In [1]:
import git
import numpy as np
import os
import tensorflow as tf
import time

repo = git.Repo('.', search_parent_directories=True)
repo.working_tree_dir

2023-06-25 05:29:47.161885: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-25 05:29:48.291509: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-06-25 05:29:48.291600: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-06-25 05:29:48.291610: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

'/home/arrow/Desktop/projects/github/ai-ml'

In [2]:
def read_files(directory, start_separator = "", end_separator = ""):
    content = ""
    for root, dirs, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    content += "\n\n" + start_separator + "\n\n"
                    content += f.read()
                    content += "\n\n" + end_separator + "\n\n"
                print(f"Read file successfully: {file_path}")
            except UnicodeDecodeError:
                print(f"Unable to read file: {file_path}")
    return content

In [3]:
# Refer to github___a-r-r-o-w@competitive-programming.md to retrieve dataset
lib_dataset_path = os.path.join(repo.working_tree_dir, "datasets/competitive-programming/lib/")
sol_dataset_path = os.path.join(repo.working_tree_dir, "datasets/competitive-programming/solutions/")

text = ""
text += read_files(lib_dataset_path, "[START_LIBRARY]", "[END_LIBRARY]")
text += read_files(sol_dataset_path, "[START_SOLUTION]", "[END_SOLUTION]")

Read file successfully: /home/arrow/Desktop/projects/github/ai-ml/datasets/competitive-programming/lib/cpp/string.hpp
Read file successfully: /home/arrow/Desktop/projects/github/ai-ml/datasets/competitive-programming/solutions/icpc/icpc-amritapuri-2021-regionals/optimal-data-transfer-again/README.md
Read file successfully: /home/arrow/Desktop/projects/github/ai-ml/datasets/competitive-programming/solutions/icpc/icpc-amritapuri-2021-regionals/optimal-data-transfer-again/main.cpp
Read file successfully: /home/arrow/Desktop/projects/github/ai-ml/datasets/competitive-programming/solutions/icpc/icpc-amritapuri-2021-regionals/maximum-peak-subsequence/brute.py
Read file successfully: /home/arrow/Desktop/projects/github/ai-ml/datasets/competitive-programming/solutions/icpc/icpc-amritapuri-2021-regionals/maximum-peak-subsequence/README.md
Read file successfully: /home/arrow/Desktop/projects/github/ai-ml/datasets/competitive-programming/solutions/icpc/icpc-amritapuri-2021-regionals/maximum-peak-

In [4]:
print(text[:1000])



[START_LIBRARY]

#include <array>
#include <cstdint>
#include <string>
#include <tuple>
#include <vector>

namespace arrow {
  namespace string {

    /**
     * @brief Check if the string `pattern` occurs in the string `text`
     *          - Time complexity: O((m - n + 1).n)
     *          - Space complexity: O(1)
     * 
     * @param text std::string
     * @param pattern std::string
     * @return int32_t -1 if match not found else index of first match in range [0, text.length)
     */
    int32_t naive_find (const std::string &text, const std::string &pattern) {
      int32_t m = text.size(), n = pattern.size();

      for (int32_t i = 0; i + n - 1 < m; ++i) {
        bool bad = false;
        
        for (int32_t j = 0; j < n; ++j) {
          if (text[i + j] != pattern[j]) {
            bad = true;
            break;
          }
        }
        
        if (not bad)
          return i;
      }

      return -1;
    }

    /**
     * @brief Z-function for a string finds a

In [5]:
vocabulary = list(sorted(set(text)))

In [6]:
chars_to_ids = tf.keras.layers.StringLookup(
    vocabulary = vocabulary,
    mask_token = None
)
ids_to_chars = tf.keras.layers.StringLookup(
    vocabulary = chars_to_ids.get_vocabulary(),
    invert = True,
    mask_token = None
)

2023-06-25 05:29:50.386706: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2023-06-25 05:29:50.386749: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [7]:
hello_world_ids = chars_to_ids(list('Hello World!'))
print(hello_world_ids)
print(tf.strings.reduce_join(ids_to_chars(hello_world_ids), axis = -1).numpy().decode())

tf.Tensor([43 72 79 79 82  3 58 82 85 79 71  4], shape=(12,), dtype=int64)
Hello World!


In [8]:
text_ids = chars_to_ids(tf.strings.unicode_split(text, 'UTF-8'))
text_dataset = tf.data.Dataset.from_tensor_slices(text_ids)

In [9]:
sequence_length = 100
sequences = text_dataset.batch(sequence_length + 1, drop_remainder = True)

In [10]:
def get_input_target(sequence):
    input_sequence = sequence[:-1]
    target_sequence = sequence[1:]
    return input_sequence, target_sequence
get_input_target('Penguin')

('Pengui', 'enguin')

In [11]:
batch_size = 64
buffer_size = 10000

dataset = sequences.map(get_input_target)
dataset = (
    dataset
        .shuffle(buffer_size)
        .batch(batch_size, drop_remainder = True)
        .prefetch(tf.data.experimental.AUTOTUNE)
)

In [12]:
vocabulary_size = len(chars_to_ids.get_vocabulary())
embedding_dim = 256
rnn_units = 160

In [13]:
class RNN(tf.keras.Model):
    def __init__(self, vocabulary_size, embedding_dim, rnn_units) -> None:
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocabulary_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units, return_sequences = True, return_state = True)
        self.dense = tf.keras.layers.Dense(vocabulary_size)
    
    def call(self, inputs, states = None, return_state = None, training = False):
        x = inputs
        x = self.embedding(x, training = training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state = states, training = training)
        x = self.dense(x, training = training)
        
        if return_state:
            return x, states
        return x

In [14]:
class CustomTrainingRNN(RNN):
    @tf.function
    def train_step(self, inputs):
        inputs, labels = inputs
        with tf.GradientTape() as tape:
            predictions = self(inputs, training=True)
            loss = self.loss(labels, predictions)
        grads = tape.gradient(loss, model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, model.trainable_variables))
        return {'loss': loss}

In [15]:
model = CustomTrainingRNN(vocabulary_size, embedding_dim, rnn_units)

In [16]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape) # B x seq_len x vocab_size

(64, 100, 120)


In [17]:
model.summary()

Model: "custom_training_rnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  30720     
                                                                 
 gru (GRU)                   multiple                  200640    
                                                                 
 dense (Dense)               multiple                  19320     
                                                                 
Total params: 250,680
Trainable params: 250,680
Non-trainable params: 0
_________________________________________________________________


In [18]:
epochs = 10
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)
model.compile(optimizer = 'adam', loss = loss)

In [19]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix,
    save_weights_only = True
)

In [20]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)
)

In [21]:
history = model.fit(dataset, epochs=epochs)

Epoch 1/10
315/315 [==============================] - 55s 164ms/step - loss: 2.7531
Epoch 2/10
315/315 [==============================] - 53s 164ms/step - loss: 1.7225
Epoch 3/10
315/315 [==============================] - 60s 188ms/step - loss: 1.3603
Epoch 4/10
315/315 [==============================] - 49s 150ms/step - loss: 1.2104
Epoch 5/10
315/315 [==============================] - 43s 135ms/step - loss: 1.1254
Epoch 6/10
315/315 [==============================] - 33s 101ms/step - loss: 1.0686
Epoch 7/10
315/315 [==============================] - 47s 146ms/step - loss: 1.0273
Epoch 8/10
315/315 [==============================] - 28s 85ms/step - loss: 0.9948
Epoch 9/10
315/315 [==============================] - 27s 85ms/step - loss: 0.9707
Epoch 10/10
315/315 [==============================] - 41s 128ms/step - loss: 0.9508


In [22]:
class Step(tf.keras.Model):
    def __init__(self, model, chars_to_ids, ids_to_chars, temperature = 1.0):
        super().__init__()
        self.model = model
        self.chars_to_ids = chars_to_ids
        self.ids_to_chars = ids_to_chars
        self.temperature = temperature
        
        skip_ids = self.chars_to_ids(['[UNK]'])[:, None]
        sparse_mask = tf.SparseTensor(
            values = [-float('inf')] * len(skip_ids),
            indices = skip_ids,
            dense_shape = [len(chars_to_ids.get_vocabulary())]
        )
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)
        
    @tf.function
    def generate_step(self, inputs, states = None):
        input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
        input_ids = self.chars_to_ids(input_chars).to_tensor()
        predicted_logits, states = self.model(
            inputs = input_ids,
            states = states,
            return_state = True
        )
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits / self.temperature
        predicted_logits = predicted_logits + self.prediction_mask
        predicted_ids = tf.random.categorical(predicted_logits, num_samples = 1)
        predicted_ids = tf.squeeze(predicted_ids, axis = -1)
        predicted_chars = self.ids_to_chars(predicted_ids)
        return predicted_chars, states

In [23]:
step_model = Step(model, chars_to_ids, ids_to_chars, 1.0)

In [24]:
input_text = '[START_SOLUTION]'
generation_length = 10000

start = time.time()
states = None
next_char = tf.constant([input_text])
result = [next_char]

for i in range(generation_length):
    next_char, states = step_model.generate_step(next_char, states)
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()

print(result[0].numpy().decode('utf-8') + '\n\n' + '-' * 80 + '\n')
print('Runtime:', end - start)

[START_SOLUTION]

# [34] Merinar Arigill

Given the dp[j] = [+1], [], [&] (auto self, int r) {
        int 3, i; };
    
    void   -->
     * 0 -> 0 house _prical = _get_arr[j] + 3);
      }
      
      for (int i = 2, n = 2 + u = 0; i >= i + 1; i < n; ++i) {
        if (2 < 10
**Output**
[nee](https://leetcode.com/problems/distrings) (Medium)
- [Count List Profinically possible sotation)
    int n = value.bage>
     *      0 reginal (TreeNode** on Number](https://leetcode.com/problems/setree](https://leetcode.com/problems/maximum-sequence) (Medium)
- [Hard-map Return `trust` to key, uphally, root nodes of the `path`. Your shorted = 10 k, nums2 is the find two Least elementY <= 505)
      if (start, houble **uler*1 * (n,'), dp[j]) / 2) / 1;
          
          for (int i = 0; i < n; ++i + 1] == c., frignt = incept;
      }

     * @brite c-cost [02].abquinary(); ++i) {
      int x = i+k;
      std::strible_memory_seconds_type: 4.2%
- likes: 4664
- dislikes: 545

<br />

### Solution

In [25]:
tf.saved_model.save(step_model, 'step_model')
step_model = tf.saved_model.load('step_model')

INFO:tensorflow:Assets written to: step_model/assets


INFO:tensorflow:Assets written to: step_model/assets
2023-06-25 05:38:33.026881: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'while' has 11 outputs but the _output_shapes attribute specifies shapes for 21 outputs. Output shapes may be inaccurate.
